<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#制作数据集(tfrecord格式)" data-toc-modified-id="制作数据集(tfrecord格式)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>制作数据集(tfrecord格式)</a></span><ul class="toc-item"><li><span><a href="#tfrecord数据定义格式(tensorflow/core/example/example_pb2.py)" data-toc-modified-id="tfrecord数据定义格式(tensorflow/core/example/example_pb2.py)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>tfrecord数据定义格式(tensorflow/core/example/example_pb2.py)</a></span></li><li><span><a href="#将数据保存为tfrecord格式" data-toc-modified-id="将数据保存为tfrecord格式-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>将数据保存为tfrecord格式</a></span><ul class="toc-item"><li><span><a href="#创建tfrecord文件" data-toc-modified-id="创建tfrecord文件-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>创建tfrecord文件</a></span></li><li><span><a href="#写入数据到tfrecord" data-toc-modified-id="写入数据到tfrecord-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>写入数据到tfrecord</a></span></li><li><span><a href="#示例-(将MNIST数据集存储到一个TFRecord文件中)" data-toc-modified-id="示例-(将MNIST数据集存储到一个TFRecord文件中)-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>示例 (将MNIST数据集存储到一个TFRecord文件中)</a></span></li></ul></li><li><span><a href="#解析tfrecord格式数据" data-toc-modified-id="解析tfrecord格式数据-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>解析tfrecord格式数据</a></span><ul class="toc-item"><li><span><a href="#从TFRecord中提取数据" data-toc-modified-id="从TFRecord中提取数据-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>从TFRecord中提取数据</a></span></li><li><span><a href="#示例-(解析TFRecord文件提取MNIST数据集)" data-toc-modified-id="示例-(解析TFRecord文件提取MNIST数据集)-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>示例 (解析TFRecord文件提取MNIST数据集)</a></span></li></ul></li><li><span><a href="#tfrecord-完整示例" data-toc-modified-id="tfrecord-完整示例-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span><a href="https://blog.csdn.net/happyhorizion/article/details/77894055" target="_blank">tfrecord 完整示例</a></a></span></li></ul></li><li><span><a href="#Tensorflow-图像处理" data-toc-modified-id="Tensorflow-图像处理-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tensorflow 图像处理</a></span><ul class="toc-item"><li><span><a href="#图像编码处理" data-toc-modified-id="图像编码处理-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>图像编码处理</a></span></li><li><span><a href="#图像大小调整" data-toc-modified-id="图像大小调整-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>图像大小调整</a></span><ul class="toc-item"><li><span><a href="#通过算法使得新的图像尽量保存原始图像上的所有信息" data-toc-modified-id="通过算法使得新的图像尽量保存原始图像上的所有信息-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>通过算法使得新的图像尽量保存原始图像上的所有信息</a></span></li><li><span><a href="#通过比例调整图像大小" data-toc-modified-id="通过比例调整图像大小-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>通过比例调整图像大小</a></span></li></ul></li></ul></li></ul></div>

## 制作数据集(tfrecord格式)
[Link1](https://blog.csdn.net/happyhorizion/article/details/77894055)

> `tfrecord` 数据文件是一种将图像数据和标签统一存储的二进制文件，能更好的利用内存，在tensorflow中快速的复制，移动，读取，存储等

> `tfrecord`文件包含了`tf.train.Example`协议缓冲区(protocol buffer，协议缓冲区包含了特征 Features)

> 写数据到TFRecords： 将数据填入到Example协议缓冲区(protocol buffer)，将协议缓冲区序列化为一个字符串， 并且通过`tf.python_io.TFRecordWriter` 写入到TFRecords文件。

> 从TFRecords读数据： 使用`tf.TFRecordReader`的`tf.parse_single_example`解析器，这个操作可以将Example协议内存块(protocol buffer)解析为张量

### tfrecord数据定义格式(tensorflow/core/example/example_pb2.py)

```python
message Example {
  Features features = 1;
};

message Features{
  map<string, Feature> featrue = 1;
};

message Feature{
  oneof kind{
    BytesList bytes_list = 1;    # 字符串
    FloatList float_list = 2;    # 实数列表
    Int64List int64_list = 3;    # 整数列表
  }
};
```

### 将数据保存为tfrecord格式
[Link1](https://blog.csdn.net/happyhorizion/article/details/77894055)

#### 创建tfrecord文件

```python
tfrecords_filename = './tfrecords/train.tfrecords'
# 创建tfrecord文件，准备写入
writer = tf.python_io.TFRecordWriter(tfrecords_filename)
```

#### 写入数据到tfrecord

从`tf.train.Example`的定义可知，`tfrecord`支持整型、浮点数和二进制三种格式，分别是
```python
* tf.train.Feature(int64_list = tf.train.Int64List(value=[int_scalar]))
* tf.train.Feature(bytes_list = tf.train.BytesList(value=[string_or_byte]))
* tf.train.Feature(bytes_list = tf.train.FloatList(value=[float_scalar]))
```

```python
for i in range(100):
    # 创建7*30，取值在0-255之间随机数组
    img_raw = np.random.random_integers(0,255,size=(7,30)) 
    img_raw = img_raw.tostring()
    example = tf.train.Example(features=tf.train.Features(
        feature={
            # 写入标签, 输入的必须是列表(list)
            'label': tf.train.Feature(
                int64_list = tf.train.Int64List(value=[i])),  
            # 写入图片数据
            'img_raw':tf.train.Feature(
                bytes_list = tf.train.BytesList(value=[img_raw]))
        }))
    # 将数据序列化后写入到tfrecord文件中
    writer.write(example.SerializeToString()) 
writer.close()
```

#### 示例 (将MNIST数据集存储到一个TFRecord文件中)

```python
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 生成整数型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 生成字符串型的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

mnist = input_data.read_data_sets("/path/to/mnist/data", one_hot=True)

images = mnist.train.images
# 获取图像分辨率, 作为Example中的一个属性
pixels = images.shape[1]
labels = mnist.train.labels
num_examples = mnist.train.num_examples

filename = "/path/to/output.tfrecords"    
writer = tf.python_io.TFRecordWriter(filename)

for index in range(num_examples):
    # 将图像矩阵转化成一个字符串
    image_raw = images[index].tostring()
    # 将一个样例转化为Example Protocol Buffer，并将所有信息写入这个数据结构
    example = tf.trian.Example(features = tf.train.Features(features = {
        'pixels' : _int64_feature(pixels),
        'labels' : _int64_feature(np.argmax(labels[index])),
        'image_raw' : _bytes_feature(image_raw)
    }))

    writer.write(example.SerializeToString())

writer.close()
```

### 解析tfrecord格式数据

从`TFRecords`文件中读取数据， 首先需要用`tf.train.string_input_producer`生成一个解析队列，之后调用`tf.TFRecordReader`的`tf.parse_single_example`解析器


#### 从TFRecord中提取数据
```python
tfrecords_filename = "train.tfrecords"

# 创建一个队列来维护输入文件列表
filename_queue = tf.train.string_input_producer([tfrecords_filename], 
                                                shuffle=True) 
# 创建一个reader来读取TFRecord文件
reader = tf.TFRecordReader()

# 从文件中读取一个样例, 返回文件名和文件
_, serialized_example = reader.read(filename_queue)   

# 取出包含image和label的feature对象
features = tf.parse_single_example(serialized_example,
                   features={
                       # TensorFlow提供了两种不同的属性解析方法；
                       # 一种方法是tf.FixedLenFeature, 该方法解析结果为一个Tensor; 
                       # 另一种是tf.VarLenFeature, 这种方法得到解析结果为SparseTensor,
                       # 用于处理稀疏数据;
                       # 解析数据的格式需要和写入数据的格式保持一致
                       'label': tf.FixedLenFeature([], tf.int64),
                       'img_raw': tf.FixedLenFeature([], tf.string),
                   }) 

# tf.decode_raw 可以将字符串解析成图像对应的像素数组
image = tf.decode_raw(features['img_raw'], tf.int64)
image = tf.reshape(image, [7,30])
label = tf.cast(features['label'], tf.int64)

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    # 创建一个协调器，管理线程
    coord=tf.train.Coordinator()
    # 启动QueueRunner, 此时文件名队列已经进队
    threads= tf.train.start_queue_runners(coord=coord)
    for i in range(20):
        example, l = sess.run([image,label])#在会话中取出image和label
        img=Image.fromarray(example, 'RGB')#这里Image是之前提到的
        img.save('./'+str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
        print(example, l)

    # 请求线程结束
    coord.request_stop()
    coord.join(threads)
```


#### 示例 (解析TFRecord文件提取MNIST数据集)

```python
import tensorflow as tf

reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["/path/to/output/tfrecords"])

# 从文件中读取一个样例
_, serialized_example = reader.read(filename_queue)

# 解析读入的一个样例 (如果要解析多个样例, 可以使用parse_example函数)
features = tf.parse_single_example(serialized_example,
            features = {
                'image_raw' : tf.FixedLenFeature([], tf.string),
                'pixels' : tf.FixedLenFeature([], tf.int64),
                'label' : tf.FixedLenFeature([], tf.int64),
            })

images = tf.decode_raw(features['image_raw'], tf.uint8)
labels = tf.cast(features['label'], tf.int32)
pixels = tf.cast(features['pixels'], tf.int32)

sess = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(10):
    image, label, pixel = sess.run([image, labels, pixels])            
```

### [tfrecord 完整示例](https://blog.csdn.net/happyhorizion/article/details/77894055)

In [ ]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import os
 
#=============================================================================#
# write images and label in tfrecord file and read them out
def encode_to_tfrecords(tfrecords_filename, data_num): 
    ''' write into tfrecord file '''
    if os.path.exists(tfrecords_filename):
        os.remove(tfrecords_filename)

    # 创建.tfrecord文件，准备写入数据    
    writer = tf.python_io.TFRecordWriter('./'+tfrecords_filename) 

    for i in range(data_num):
        img_raw = np.random.randint(0,255,size=(56,56))
        img_raw = img_raw.tostring()
        example = tf.train.Example(features=tf.train.Features(
            feature={
                'label': tf.train.Feature(int64_list = tf.train.Int64List(value=[i])),     
                'img_raw':tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_raw]))
            }))
        writer.write(example.SerializeToString()) 

    writer.close()

    
def decode_from_tfrecords(filename_queue, is_batch):
    
    reader = tf.TFRecordReader()
    # 返回文件名和文件
    _, serialized_example = reader.read(filename_queue)   
    # 取出包含image和label的feature对象
    features = tf.parse_single_example(serialized_example,
                                    features={
                                        'label': tf.FixedLenFeature([], tf.int64),
                                        'img_raw' : tf.FixedLenFeature([], tf.string),
                                    })  
    image = tf.decode_raw(features['img_raw'],tf.int64)
    image = tf.reshape(image, [56,56])
    label = tf.cast(features['label'], tf.int64)

    if is_batch:
        batch_size = 3
        num_threads = 3
        min_after_dequeue = 10
        # 需要注意: capacity >= min_after_dequeue + num_threads * batch_size
        capacity = min_after_dequeue + num_threads * batch_size
        image, label = tf.train.shuffle_batch([image, label],
                                            batch_size=batch_size, 
                                            num_threads=num_threads, 
                                            capacity=capacity,
                                            min_after_dequeue=min_after_dequeue)
    return image, label
 
#=============================================================================#
 
if __name__=='__main__':
    
    # make train.tfrecord
    train_filename = "train.tfrecords"
    encode_to_tfrecords(train_filename,100)
    
    # make test.tfrecord
    test_filename = 'test.tfrecords'
    encode_to_tfrecords(test_filename,10)

    filename_queue = tf.train.string_input_producer([train_filename],num_epochs=None)
    train_image, train_label = decode_from_tfrecords(filename_queue, is_batch=True)

    filename_queue = tf.train.string_input_producer([test_filename],num_epochs=None) 
    test_image, test_label = decode_from_tfrecords(filename_queue, is_batch=True)
    with tf.Session() as sess: #开始一个会话
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        coord=tf.train.Coordinator()
        threads= tf.train.start_queue_runners(coord=coord)

        try:
            # while not coord.should_stop():
            for i in range(2):
                example, l = sess.run([train_image,train_label])#在会话中取出image和label
                print('train:')
                print(example, l) 
                texample, tl = sess.run([test_image, test_label])
                print('test:')
                print(texample,tl)
        except tf.errors.OutOfRangeError:
            print('Done reading')
        finally:
            coord.request_stop()

        coord.request_stop()
        coord.join(threads)

## Tensorflow 图像处理

### 图像编码处理
> 一张RGB色彩模式的图像可以看成是一个三维矩阵，矩阵中的每一个数字表示了图像上不同位置，不同颜色的亮度；然而图像在存储时并不是直接记录这些矩阵中的数字，而是记录了经过压缩编码之后的结果。所以要将一张图像还原成一个三维矩阵，需要解码的过程

```python
import matplotlib.pyplot as plt
import tensorflow as tf

# 读取图像的原始数据
image_raw_data = tf.gfile.FastGFile("/path/to/picture", "r").read()

with tf.Session() as sess:
    # 将图像使用jpeg的格式解码从而得到图像对应的三维矩阵
    # tensorflow还提供了tf.image.decode_png函数
    # 对png格式的图像进行解码; 解码之后的结果为一个张量
    img_data = tf.image.decode_jpeg(image_raw_data)

    # 输出解码之后的三维矩阵
    print img_data.eval()

# 使用pyplot工具可视化得到的结果
plt.imshow(img_data.eval())
plt.show()

# 将数据的类型转化成实数
img_data = tf.image.convert_image_dtype(img_data, dtype=tf.float32)

# 将图像的三维矩阵重新按照jpeg的格式编码写入文件
encoded_image = tf.image.encode_jpeg(img_data)
with tf.gfile.GFile("/path/to/output", "wb") as f:
    f.write(encoded_image.eval())
```

### 图像大小调整
> 一般来说，网络上获取的图像大小是不固定的，但神经网络输入节点的个数是固定的；所以在将图像的像素作文输入提供给神经网络之前，需要先将图像的大小统一

#### 通过算法使得新的图像尽量保存原始图像上的所有信息

* 如果原始图像的尺寸大于目标图像, 那么会自动截取原始图像居中的部分
* 如果目标图像大于原始图像, 会自动在原始图像的四周填充全0背景

```python
import matplotlib.pyplot as plt;
import tensorflow as tf;
 
image_raw_data_jpg = tf.gfile.FastGFile('11.jpg', 'r').read()
 
with tf.Session() as sess:
	img_data_jpg = tf.image.decode_jpeg(image_raw_data_jpg)
	img_data_jpg = tf.image.convert_image_dtype(img_data_jpg, dtype=tf.float32)
    
    """
    resize_images(images,
                  size,                            # 调整后图像的大小
                  method=ResizeMethod.BILINEAR,    # 调整图像的算法
                  align_corners=False,
                  preserve_aspect_ratio=False):
    """
    
    # method=0; 双线性插值法(Bilinear interpolation)
	resize_0 = tf.image.resize_images(img_data_jpg, (500, 500), method=0)
    # method=1; 最近邻居法(Nearest neighbor interpolation)
	resize_1 = tf.image.resize_images(img_data_jpg, (500, 500), method=1)
    # method=2; 双三次插值法(Bicubic interpolation)
	resize_2 = tf.image.resize_images(img_data_jpg, (500, 500), method=2)
    # method=3; 面积插值法(Area interpolation)
	resize_3 = tf.image.resize_images(img_data_jpg, (500, 500), method=3)
	
    print resize_0.get_shape    # (500, 500, ?)  图像的深度在没明确设置之前是问号
 
	plt.figure(0)
	plt.imshow(resize_0.eval())
	plt.figure(1)
	plt.imshow(resize_1.eval())
	plt.figure(2)
	plt.imshow(resize_2.eval())
	plt.figure(3)
	plt.imshow(resize_3.eval())
 
	plt.show()

```

Tensorflow 还提供了API对图像进行裁剪或者填充
```python
import cv2
import tensorflow as tf

img = cv2.imread("/path/to/image")

"""
tf.image.resize_image_with_crop_or_pad(
    image,
    target_height,
    target_width
)

Args:
    image: 4-D Tensor of shape `[batch, height, width, channels]` or
           3-D Tensor of shape `[height, width, channels]`.
    target_height: Target height.
    target_width: Target width.
"""
img2 = tf.image.resize_image_with_crop_or_pad(img, 
                                            target_height=200, 
                                            target_width=200)

with tf.Session() as sess:
    img2 = sess.run(img2)
    cv2.imshow("image2", img2)       
    cv2.waitKey(0)          
```



#### 通过比例调整图像大小

通过tf.image.central_crop函数可以按比例裁剪图像
```python
"""
tf.image.central_crop(
    image,
    central_fraction
)

Args:
    image: Either a 3-D float Tensor of shape [height, width, depth], or a 4-D Tensor of shape [batch_size, height, width, depth].
    central_fraction: float (0, 1], fraction of size to crop
"""

central_cropped = tf.image.central.crop(img_data, 0.5)
```